# 人像分割及模型量化实战
本项目基于PaddleSeg核心分割网络，提供针对人像分割场景从预训练模型、Fine-tune、视频分割预测部署。应用HumanSeg-lite模型超轻量级人像分割模型，支持移动端场景的实时分割。

## 场景应用
将人像与背景分离，可用于背景替换，如影视特效制作，全民小视频等APP中的人像分割并替换背景特效等

## PaddleSeg 安装

In [1]:
# 若在本地环境运行，使用git下载PaddleSeg代码
#! git clone https://github.com/PaddlePaddle/PaddleSeg.git

# 为快速，此处我们已下载PaddleSeg代码，解压后可直接执行后续代码
!unzip PaddleSeg.zip

Archive:  PaddleSeg.zip
replace PaddleSeg/.pre-commit-config.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
# 进入人像分割目录
%cd PaddleSeg/contrib/HumanSeg


## 环境依赖
* Python == 3.5/3.6/3.7
* PaddlePaddle >= 1.7.1
通过以下命令安装python包依赖，确保至少执行过一次以下命令

In [3]:
!pip install paddleslim

## 预训练模型
HumanSeg开放了在大规模人像数据上训练的三个预训练模型，满足多种使用场景的需求
| 模型类型 | Checkpoint | Inference Model | Quant Inference Model | 备注 |
| --- | --- | --- | ---| --- |
| HumanSeg-server  | [humanseg_server_ckpt](https://paddleseg.bj.bcebos.com/humanseg/models/humanseg_server_ckpt.zip) | [humanseg_server_inference](https://paddleseg.bj.bcebos.com/humanseg/models/humanseg_server_inference.zip) | -- | 高精度模型，适用于服务端GPU且背景复杂的人像场景， 模型结构为Deeplabv3+/Xcetion65, 输入大小（512， 512） |
| HumanSeg-mobile | [humanseg_mobile_ckpt](https://paddleseg.bj.bcebos.com/humanseg/models/humanseg_mobile_ckpt.zip) | [humanseg_mobile_inference](https://paddleseg.bj.bcebos.com/humanseg/models/humanseg_mobile_inference.zip) | [humanseg_mobile_quant](https://paddleseg.bj.bcebos.com/humanseg/models/humanseg_mobile_quant.zip) | 轻量级模型, 适用于移动端或服务端CPU的前置摄像头场景，模型结构为HRNet_w18_samll_v1，输入大小（192， 192）  |
| HumanSeg-lite | [humanseg_lite_ckpt](https://paddleseg.bj.bcebos.com/humanseg/models/humanseg_lite_ckpt.zip) | [humanseg_lite_inference](https://paddleseg.bj.bcebos.com/humanseg/models/humanseg_lite_inference.zip) |  [humanseg_lite_quant](https://paddleseg.bj.bcebos.com/humanseg/models/humanseg_lite_quant.zip) | 超轻量级模型, 适用于手机自拍人像，且有移动端实时分割场景， 模型结构为优化的ShuffleNetV2，输入大小（192， 192） |


模型性能

| 模型 | 模型大小 | 计算耗时 |
| --- | --- | --- |
|humanseg_server_inference| 158M | - |
|humanseg_mobile_inference | 5.8 M | 42.35ms |
|humanseg_mobile_quant | 1.6M | 24.93ms |
|humanseg_lite_inference | 541K | 17.26ms |
|humanseg_lite_quant | 187k | 11.89ms |

计算耗时运行环境： 小米，cpu：骁龙855， 内存：6GB， 图片大小：192*192)

**NOTE:**
其中Checkpoint为模型权重，用于Fine-tuning场景。

* Inference Model和Quant Inference Model为预测部署模型，包含`__model__`计算图结构、`__params__`模型参数和`model.yaml`基础的模型配置信息。

* 其中Inference Model适用于服务端的CPU和GPU预测部署，Qunat Inference Model为量化版本，适用于通过Paddle Lite进行移动端等端侧设备部署。更多Paddle Lite部署说明查看[Paddle Lite文档](https://paddle-lite.readthedocs.io/zh/latest/)



## 下载预训练模型

In [4]:
!python data/download_data.py
!python pretrained_weights/download_pretrained_weights.py

## 下载测试数据
[supervise.ly](https://supervise.ly/)发布人像分割数据集**Supervisely Persons**, 从中随机抽取一小部分并转化成PaddleSeg可直接加载数据格式。通过运行以下代码进行快速下载，其中包含手机前置摄像头的人像测试视频`video_test.mp4`.

In [5]:
!python data/download_data.py

## 快速体验视频流人像分割
分割过程中，结合DIS（Dense Inverse Search-basedmethod）光流算法预测结果与分割结果，改善视频流人像分割。

支持对摄像头获取的视频流进行实时分割，也支持对视频进行分割。在AIStudio运行仅支持对视频进行分割。


In [6]:
# 通过电脑摄像头进行时分割处理
# !python video_infer.py --model_dir pretrained_weights/humanseg_lite_inference

#  对人像视频进行分割处理
!python video_infer.py --model_dir pretrained_weights/humanseg_lite_inference --video_path data/video_test.mp4

视频分割结果如下：

<img src="https://paddleseg.bj.bcebos.com/humanseg/data/video_test.gif" width="10%" height="10%"><img src="https://paddleseg.bj.bcebos.com/humanseg/data/result.gif" width="10%" height="10%">


**NOTE:**

视频分割处理时间需要几分钟，请耐心等待。

## 训练
使用下述命令基于与训练模型进行Fine-tuning，请确保选用的模型结构`model_type`与模型参数`pretrained_weights`匹配。

In [7]:
!python train.py --model_type HumanSegMobile \
--save_dir output/ \
--data_dir data/mini_supervisely \
--train_list data/mini_supervisely/train.txt \
--val_list data/mini_supervisely/val.txt \
--pretrained_weights pretrained_weights/humanseg_mobile_ckpt \
--batch_size 8 \
--learning_rate 0.001 \
--num_epochs 10 \
--image_shape 192 192

其中参数含义如下：
* `--model_type`: 模型类型，可选项为：HumanSegServer、HumanSegMobile和HumanSegLite
* `--save_dir`: 模型保存路径
* `--data_dir`: 数据集路径
* `--train_list`: 训练集列表路径
* `--val_list`: 验证集列表路径
* `--pretrained_weights`: 预训练模型路径
* `--batch_size`: 批大小
* `--learning_rate`: 初始学习率
* `--num_epochs`: 训练轮数
* `--image_shape`: 网络输入图像大小（w, h）

更多命令行帮助可运行下述命令进行查看：

In [8]:
!python train.py --help

**NOTE**
可通过更换`--model_type`变量与对应的`--pretrained_weights`使用不同的模型快速尝试。

## 评估
使用下述命令进行评估

In [9]:
!python val.py --model_dir output/best_model \
--data_dir data/mini_supervisely \
--val_list data/mini_supervisely/val.txt \
--image_shape 192 192

其中参数含义如下：
* `--model_dir`: 模型路径
* `--data_dir`: 数据集路径
* `--val_list`: 验证集列表路径
* `--image_shape`: 网络输入图像大小（w, h）

## 预测
使用下述命令进行预测

In [10]:
!python infer.py --model_dir output/best_model \
--data_dir data/mini_supervisely \
--test_list data/mini_supervisely/test.txt \
--image_shape 192 192

其中参数含义如下：
* `--model_dir`: 模型路径
* `--data_dir`: 数据集路径
* `--test_list`: 测试集列表路径
* `--image_shape`: 网络输入图像大小（w, h）

### 结果展示
图片预测结果默认保存在output/result目录中， 分别保存有叠加图、scoremap和人像扣取结果。


In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
# 定义显示函数
def display(img_dir, title, flag=0):
    plt.figure(figsize=(15, 15))
    for i in range(len(title)):
        if flag:
            plt.subplot(len(img_dir), 1, i+1)
        else:
            plt.subplot(1, len(img_dir), i+1)
        plt.title(title[i])
        img = plt.imread(img_dir[i])
        plt.imshow(img)
        plt.axis('off')
    plt.show()

In [18]:
# 预测结果展示
added_path = "output/result/added/Images/pexels-photo-573300_KrLVVmXzru.png"
score_path = "output/result/scoremap/Images/pexels-photo-573300_KrLVVmXzru.png"
mat_path = "output/result/mat/Images/pexels-photo-573300_KrLVVmXzru.png"
display([added_path, score_path, mat_path], ['added Image', 'scoremap', 'mat'])

## 模型导出

In [14]:
!python export.py --model_dir output/best_model \
--save_dir output/export

其中参数含义如下：
* `--model_dir`: 模型路径
* `--save_dir`: 导出模型保存路径

## 离线量化

In [15]:
!python quant_offline.py --model_dir output/best_model \
--data_dir data/mini_supervisely \
--quant_list data/mini_supervisely/val.txt \
--save_dir output/quant_offline \
--image_shape 192 192

其中参数含义如下：
* `--model_dir`: 待量化模型路径
* `--data_dir`: 数据集路径
* `--quant_list`: 量化数据集列表路径，一般直接选择训练集或验证集
* `--save_dir`: 量化模型保存路径
* `--image_shape`: 网络输入图像大小（w, h）

## 在线量化
利用float训练模型进行在线量化。

In [16]:
!python quant_online.py --model_type HumanSegMobile \
--save_dir output/quant_online \
--data_dir data/mini_supervisely \
--train_list data/mini_supervisely/train.txt \
--val_list data/mini_supervisely/val.txt \
--pretrained_weights output/best_model \
--batch_size 2 \
--learning_rate 0.001 \
--num_epochs 2 \
--image_shape 192 192

其中参数含义如下：
* `--model_type`: 模型类型，可选项为：HumanSegServer、HumanSegMobile和HumanSegLite
* `--save_dir`: 模型保存路径
* `--data_dir`: 数据集路径
* `--train_list`: 训练集列表路径
* `--val_list`: 验证集列表路径
* `--pretrained_weights`: 预训练模型路径,
* `--batch_size`: 批大小
* `--learning_rate`: 初始学习率
* `--num_epochs`: 训练轮数
* `--image_shape`: 网络输入图像大小（w, h）

**NOTE:**
在线量化的效果不一定优于离线量化。离线量化操作简单，对于量化不甚了解的同学推荐直接使用离线量化即可。